In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train[..., None]/255.0, x_test[..., None]/255.0

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=1)

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 36s 18ms/step - accuracy: 0.9108 - loss: 0.2998


In [2]:
model.export("saved_model/mnist_cnn")

INFO:tensorflow:Assets written to: saved_model/mnist_cnn\assets


INFO:tensorflow:Assets written to: saved_model/mnist_cnn\assets


Saved artifact at 'saved_model/mnist_cnn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1724157382672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1724157384592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1724157384208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1724157383440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1724157385168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1724157384784: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [29]:
!docker pull tensorflow/serving

'docker' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
docker run -p 8501:8501 --name tf_serving_mnist \
  --mount type=bind,source=$(pwd)/saved_model/mnist_cnn,target=/models/mnist_cnn \
  -e MODEL_NAME=mnist_cnn -t tensorflow/serving

SyntaxError: invalid syntax (2703837496.py, line 1)

In [24]:
import requests
import json
import numpy as np

sample = x_test[0:1].tolist()
data = json.dumps({"signature_name": "serving_default", "instances": sample})
headers = {"content-type": "application/json"}
url = "http://localhost:8501/v1/models/mnist_cnn:predict"
response = requests.post(url, data=data, headers=headers)

InvalidSchema: No connection adapters were found for 'D:/mnist_test.csv'

In [ ]:
print("Predicted class probabilities:", response.json())
print("Predicted class:", np.argmax(response.json()['predictions'][0]))